In [18]:
# Подключение библиотек
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyproj import Transformer

In [19]:
# Преобразование из формата .npz в .hdf, удаление лишних данных, установление даты в качестве индекса
data = np.load('C:/Users/Maks/Desktop/Jupyter/2012_thunderbolts_clastered.npz')
df = pd.DataFrame(data['strikes']).drop('tail', axis=1).set_index('date')
df.to_hdf('2012_thunderbolts_clastered.h5', key='strikes', mode='w', complevel=4)
hf = pd.read_hdf('2012_thunderbolts_clastered.h5', 'strikes')

In [20]:
# Создание DataFrame "clusters" с информацией о номере кластера, его длительности, количестве разрядов и покрываемой площади
clusters = pd.DataFrame(columns=['clnb', 'durn', 'amnt', 'area'])
for i in range(hf.clnb.max()):
    clst = hf.loc[hf['clnb'] == i+1] 
    amount = len(clst) # Кол-во разрядов в кластере
    if amount > 0:
        Date_min = clst.index.min(); Date_max = clst.index.max()
        Date_diff = Date_max - Date_min # Длительность кластера
        min_lat = clst.lat.min(); min_lon = clst.lon.min()
        max_lat = clst.lat.max(); max_lon = clst.lon.max()
        average_lat = (min_lat + max_lat)/2; average_lon = (min_lon + max_lon)/2
        zone = round((average_lon + 180) / 6) # Определение UTM-зоны
        if average_lat >= 0:
            epsg_code = 32600 + zone  # Северное полушарие
        else:
            epsg_code = 32700 + zone  # Южное полушарие
        tr = Transformer.from_crs('EPSG:4326', 'EPSG:' + str(epsg_code)) # Преобразование координат из WGS84 (широта, долгота) в UTM (метры)
        y1, x1 = tr.transform(min_lat, min_lon)
        y2, x2 = tr.transform(max_lat, max_lon)
        width = np.abs(x2 - x1)
        height = np.abs(y2 - y1)
        s = width * height # Покрываемая кластером площадь
        clusters_1 = pd.DataFrame({'clnb': [i+1], 'durn': [Date_diff], 'amnt': [amount], 'area': [s]})
        clusters_sum = pd.concat([clusters, clusters_1])
        clusters = clusters_sum
clusters = clusters.set_index('clnb')
clusters

C:\Temp\ipykernel_2960\19057738.py:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  clusters_sum = pd.concat([clusters, clusters_1])


,durn,amnt,area
clnb,,,
2,0 days 01:11:06,24,6.941394e+09
3,0 days 01:42:43,875,3.714398e+10
4,0 days 03:48:16,280,3.669214e+10
6,0 days 00:22:47,6,3.436433e+08
7,0 days 03:47:47,717,7.568382e+10
...,...,...,...
5432,0 days 00:58:11,23,1.879951e+09
5433,0 days 01:01:27,9,5.428085e+08
5434,0 days 00:19:06,6,5.705392e+08


In [ ]:
# Построение графиков и сохранение в отдельные файлы
for j in range(hf.clnb.max()):
  Data_lon = []
  Data_lat = []
  Data_amp = [] 
  for i in range(len(hf)):
    if hf.iloc[i, 3] == j+1:
      Data_lon.append(hf.iloc[i, 1])
      Data_lat.append(hf.iloc[i, 0])
      Data_amp.append(np.abs(hf.iloc[i, 2]))
  if len(Data_lon) > 0: 
    plt.scatter(Data_lon, Data_lat, c=Data_amp, vmin=0, vmax=hf.amp.max())
    plt.title('clnb '+str(j))
    plt.xlabel('lon'); plt.ylabel('lat')
    plt.grid()
    cb = plt.colorbar()
    cb.set_label('amp')
    plt.savefig("C:/Users/Maks/Desktop/Jupyter/plot/thunder_plot_clnb_" + str(j) + ".png")
    plt.close()